# Summary  
final datasets are df2022,df2021,df2020,df2019,df2018  
these datasets are combined from TS dataset and On-Off dataset  
**not export yet, need to be exported to be used in main analysis**  

In [1]:
import pandas as pd
import numpy as np

# 1.load TS data

In [2]:
raw_TSdata2022 = "data/00-raw/TS_datas/TS-2022.csv"
raw_TSdata2021 = "data/00-raw/TS_datas/TS-2021.csv"
raw_TSdata2020 = "data/00-raw/TS_datas/TS-2020.csv"
raw_TSdata2019 = "data/00-raw/TS_datas/TS-2019.csv"
raw_TSdata2018 = "data/00-raw/TS_datas/TS-2018.csv"

In [3]:
raw_TS2022 = pd.read_csv(raw_TSdata2022)
raw_TS2021 = pd.read_csv(raw_TSdata2021)
raw_TS2020 = pd.read_csv(raw_TSdata2020)
raw_TS2019 = pd.read_csv(raw_TSdata2019)
raw_TS2018 = pd.read_csv(raw_TSdata2018)
raw_TS2022.head()

,Rk,Player,Age,Team,Pos,G,GS,MP,PER,TS%,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Awards,Player-additional
0,1.0,Mikal Bridges,25.0,PHO,SF,82.0,82.0,2854.0,14.4,0.627,...,5.2,3.7,8.9,0.150,0.5,1.1,1.6,2.6,DPOY-2DEF1,bridgmi01
1,2.0,Miles Bridges,23.0,CHO,PF,80.0,80.0,2837.0,17.9,0.593,...,5.0,2.3,7.2,0.123,1.8,-0.3,1.5,2.5,NaN,bridgmi02
2,3.0,DeMar DeRozan,32.0,CHI,PF,76.0,76.0,2743.0,23.1,0.590,...,7.0,1.8,8.8,0.154,3.4,-0.9,2.5,3.1,MVP-10ASNBA2,derozde01
3,4.0,Jayson Tatum,23.0,BOS,SF,76.0,76.0,2731.0,21.8,0.578,...,5.0,4.6,9.6,0.169,4.3,0.6,5.0,4.8,MVP-6ASNBA1,tatumja01
4,5.0,Saddiq Bey,22.0,DET,SF,82.0,82.0,2704.0,14.0,0.529,...,2.4,1.6,4.0,0.071,0.9,-1.2,-0.3,1.2,NaN,beysa01


In [4]:
raw_TS2022.shape

(813, 30)

In [5]:
columns = raw_TS2022.columns
column_sets = [raw_TS2021.columns, raw_TS2020.columns, raw_TS2019.columns, raw_TS2018.columns]


In [6]:
def check_columns(columns, column_sets):
    i = 2021
    for column in column_sets:
        i = i - 1
        if  (columns != column).any():
            print(f"column{i} don't match")
    print("all columns match")

In [7]:
check_columns(columns, column_sets)
print(columns)

all columns match
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Awards', 'Player-additional'],
      dtype='object')


## 2.filter out the columns  

In [8]:
keep_columns = ['Player', 'Team','G', 'MP', 'TS%']

In [9]:
TS2022 = raw_TS2022[keep_columns]
TS2021 = raw_TS2021[keep_columns]
TS2020 = raw_TS2020[keep_columns]
TS2019 = raw_TS2019[keep_columns]
TS2018 = raw_TS2018[keep_columns]
# df2022.head(10)
# df2022.dtypes

## 3.drop missing data

In [10]:
TS2022 = TS2022.dropna()
TS2021 = TS2021.dropna()
TS2020 = TS2020.dropna()
TS2019 = TS2019.dropna()
TS2018 = TS2018.dropna()

In [11]:
# TS2022.isna().any(axis=1).sum()
# TS2021.isna().any(axis=1).sum()
# TS2018.isna().any(axis=1).sum()

# 4.Handle duplicate 

## 4a.duplicated row example

In [12]:
players2022 = TS2022['Player'].unique()
# print("unique players count", players2022.shape)
# print("players count", TS2022['Player'].shape)
TS2022.query('Player=="Tyrese Haliburton"')
# TS2022['Team'].unique()

,Player,Team,G,MP,TS%
5,Tyrese Haliburton,2TM,77.0,2695.0,0.595
6,Tyrese Haliburton,SAC,51.0,1757.0,0.575
7,Tyrese Haliburton,IND,26.0,938.0,0.629


## 4b.handle duplicate
rule: keep the row with 'Team' = xTM when there are duplicated rows  
x: any number ex: 2TM, 3TM

In [13]:
#Not Use
def combine_rows(df):
    players = df['Player'].unique()
    rows = []
    for player in players:
        #if(f'(df["Player"]=="{player}").sum() > 1'):
        player_df = df.query('Player == @player')
        #Not USE:filter out all the rows of this player, add the combined row back later
        #df = df[f"df['Player'] != '{player}'"]
        #teams = '/'.join(player_df['Team'].unique())
        total_G = player_df['G'].sum()
        total_MP = player_df['MP'].sum()
        average_TS = (player_df['TS%'].mean())
        rows.append({
                    'Player':player,
                    'G':total_G, 
                    'MP':total_MP, 
                    'TS%':average_TS
                    })
    return pd.DataFrame(rows)

In [14]:
def handle_dup(df):
    df = (df.groupby('Player', as_index=False)
          .apply(lambda x: 
              x[x['Team'].astype(str).str.match(r'^\d+TM$')].iloc[0]
              if x['Team'].astype(str).str.match(r'^\d+TM$').any()
                 else x.iloc[0],
         include_groups=False))
    return df

In [15]:
TS2022 = handle_dup(TS2022)
TS2021 = handle_dup(TS2021)
TS2020 = handle_dup(TS2020)
TS2019 = handle_dup(TS2019)
TS2018 = handle_dup(TS2018)


In [16]:
#check duplicate
# print(TS2022['Player'].duplicated().any())
# print(TS2021['Player'].duplicated().any())
# print(TS2020['Player'].duplicated().any())
# print(TS2019['Player'].duplicated().any())
# print(TS2018['Player'].duplicated().any())
# TS2022.query('Player=="Tyrese Haliburton"')
# TS2022['Team'].unique()

##### note  
we can combine rows in a single dataframe, and we can filter columns in a single dataframe  
should I combine all the dataframe into a big one and perform filter and combining rows?

# 4.5.add MPG column

In [17]:
TS2022.head()
TS2022['MPG'] = TS2022['MP'] / TS2022['G']
TS2021['MPG'] = TS2021['MP'] / TS2021['G']
TS2020['MPG'] = TS2020['MP'] / TS2020['G']
TS2019['MPG'] = TS2019['MP'] / TS2019['G']
TS2018['MPG'] = TS2018['MP'] / TS2018['G']


# 5.load on/off data

In [18]:
raw_dataOF2022 = "data/00-raw/OF_datas/on-off-2022.csv"
raw_dataOF2021 = "data/00-raw/OF_datas/on-off-2021.csv"
raw_dataOF2020 = "data/00-raw/OF_datas/on-off-2020.csv"
raw_dataOF2019 = "data/00-raw/OF_datas/on-off-2019.csv"
raw_dataOF2018 = "data/00-raw/OF_datas/on-off-2018.csv"


In [19]:
raw_OFdf2022 = pd.read_csv(raw_dataOF2022)
raw_OFdf2021 = pd.read_csv(raw_dataOF2021)
raw_OFdf2020 = pd.read_csv(raw_dataOF2020)
raw_OFdf2019 = pd.read_csv(raw_dataOF2019)
raw_OFdf2018 = pd.read_csv(raw_dataOF2018)


## 5a.filter columns

In [20]:
raw_OFdf2022.head()
raw_OFdf2022.columns
raw_OFdf2022['Unnamed: 3'].unique()
raw_OFdf2021.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Position Estimate',
       'Position Estimate.1', 'Position Estimate.2', 'Position Estimate.3',
       'Position Estimate.4', '+/- Per 100 Poss', '+/- Per 100 Poss.1',
       'Turnovers', 'Turnovers.1', 'Fouls Committed', 'Fouls Committed.1',
       'Fouls Drawn', 'Fouls Drawn.1', 'Misc.', 'Misc..1', 'Misc..2',
       'Unnamed: 24', '-additional'],
      dtype='object')

In [21]:
raw_OFdf2021.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Position Estimate,Position Estimate.1,...,Turnovers.1,Fouls Committed,Fouls Committed.1,Fouls Drawn,Fouls Drawn.1,Misc.,Misc..1,Misc..2,Unnamed: 24,-additional
0,Rk,Player,Age,Team,Pos,G,GS,MP,PG%,SG%,...,LostBall,Shoot,Off.,Shoot,Off.,PGA,And1,Blkd,Awards,-9999
1,1,Julius Randle,26,NYK,PF,71,71,2667,0,0,...,71,73,43,180,7,1092,49,70,MVP-8MIP-1ASNBA2,randlju01
2,2,RJ Barrett,20,NYK,SG,72,72,2511,5,79,...,35,85,14,133,13,520,28,86,NaN,barrerj01
3,3,Nikola Jokić,25,DEN,C,72,72,2488,0,0,...,63,102,16,145,5,1410,38,62,MVP-1ASNBA1,jokicni01
4,4,Buddy Hield,28,SAC,SG,71,71,2433,0,44,...,33,91,5,21,4,579,5,33,NaN,hieldbu01


In [22]:
#'Unname: 1' is the Player column
#'Unname: 3' is the Team column
#'+/- Per 100 Poss.1' is the on/off column that we need
OF_keep_columns = ['Unnamed: 1','Unnamed: 3', '+/- Per 100 Poss.1']

OFdf2022 = raw_OFdf2022[OF_keep_columns]
OFdf2022.columns = ['Player','Team', 'On-Off']
OFdf2022 = OFdf2022.drop(index=0)

OFdf2021 = raw_OFdf2021[OF_keep_columns]
OFdf2021.columns = ['Player', 'Team','On-Off']
OFdf2021 = OFdf2021.drop(index=0)

OFdf2020 = raw_OFdf2020[OF_keep_columns]
OFdf2020.columns = ['Player', 'Team','On-Off']
OFdf2020 = OFdf2020.drop(index=0)

OFdf2019 = raw_OFdf2019[OF_keep_columns]
OFdf2019.columns = ['Player', 'Team','On-Off']
OFdf2019 = OFdf2019.drop(index=0)

OFdf2018 = raw_OFdf2018[OF_keep_columns]
OFdf2018.columns = ['Player', 'Team','On-Off']
OFdf2018 = OFdf2018.drop(index=0)

## 5b.drop missing data

In [23]:
OFdf2022 = OFdf2022.dropna()
OFdf2021 = OFdf2021.dropna()
OFdf2020 = OFdf2020.dropna()
OFdf2019 = OFdf2019.dropna()
OFdf2018 = OFdf2018.dropna()

In [24]:
print(OFdf2022.isna().any(axis=1).sum())
print(OFdf2021.isna().any(axis=1).sum())
print(OFdf2020.isna().any(axis=1).sum())
print(OFdf2019.isna().any(axis=1).sum())
print(OFdf2018.isna().any(axis=1).sum())

0
0
0
0
0


## 5c.handle duplicate

In [25]:
# print(OFdf2022['Player'].duplicated().any())

In [26]:
OF2022 = handle_dup(OFdf2022)
OF2021 = handle_dup(OFdf2021)
OF2020 = handle_dup(OFdf2020)
OF2019 = handle_dup(OFdf2019)
OF2018 = handle_dup(OFdf2018)

In [27]:
# print(OF2022['Player'].duplicated().any())
# print(OF2021['Player'].duplicated().any())
# print(OF2020['Player'].duplicated().any())
# print(OF2019['Player'].duplicated().any())
# print(OF2018['Player'].duplicated().any())


In [28]:
OF2022.head()

,Player,Team,On-Off
0,Aaron Gordon,DEN,9.0
1,Aaron Henry,PHI,-54.1
2,Aaron Holiday,2TM,-3.2
3,Aaron Nesmith,BOS,-3.8
4,Aaron Wiggins,OKC,1.5


# 6.Combine TS dataset and On/Off dataset

In [29]:
TS2022.head()

,Player,Team,G,MP,TS%,MPG
0,Aaron Gordon,DEN,75.0,2376.0,0.602,31.680000
1,Aaron Henry,PHI,6.0,17.0,0.200,2.833333
2,Aaron Holiday,2TM,63.0,1021.0,0.544,16.206349
3,Aaron Nesmith,BOS,52.0,574.0,0.507,11.038462
4,Aaron Wiggins,OKC,50.0,1209.0,0.556,24.180000


In [30]:
OF2022.head()

,Player,Team,On-Off
0,Aaron Gordon,DEN,9.0
1,Aaron Henry,PHI,-54.1
2,Aaron Holiday,2TM,-3.2
3,Aaron Nesmith,BOS,-3.8
4,Aaron Wiggins,OKC,1.5


In [31]:
df2022 = pd.merge(TS2022, OF2022, on='Player', how='inner')
df2021 = pd.merge(TS2021, OF2021, on='Player', how='inner')
df2020 = pd.merge(TS2020, OF2020, on='Player', how='inner')
df2019 = pd.merge(TS2019, OF2019, on='Player', how='inner')
df2018 = pd.merge(TS2018, OF2018, on='Player', how='inner')

In [32]:
df2019.head()

,Player,Team_x,G,MP,TS%,MPG,Team_y,On-Off
0,Aaron Gordon,ORL,78.0,2633.0,0.538,33.756410,ORL,2.6
1,Aaron Holiday,IND,50.0,646.0,0.518,12.920000,IND,5.1
2,Abdel Nader,OKC,61.0,694.0,0.522,11.377049,OKC,-15.3
3,Al Horford,BOS,68.0,1973.0,0.605,29.014706,BOS,2.7
4,Al-Farouq Aminu,POR,81.0,2292.0,0.568,28.296296,POR,10.0


## 6.1 remove team columns

In [33]:
df2022 = df2022[['Player','G','MP','TS%','MPG','On-Off']]
df2021 = df2021[['Player','G','MP','TS%','MPG','On-Off']]
df2020 = df2020[['Player','G','MP','TS%','MPG','On-Off']]
df2019 = df2019[['Player','G','MP','TS%','MPG','On-Off']]
df2018 = df2018[['Player','G','MP','TS%','MPG','On-Off']]

In [34]:
df2022.head()

,Player,G,MP,TS%,MPG,On-Off
0,Aaron Gordon,75.0,2376.0,0.602,31.680000,9.0
1,Aaron Henry,6.0,17.0,0.200,2.833333,-54.1
2,Aaron Holiday,63.0,1021.0,0.544,16.206349,-3.2
3,Aaron Nesmith,52.0,574.0,0.507,11.038462,-3.8
4,Aaron Wiggins,50.0,1209.0,0.556,24.180000,1.5


# 7.export datasets

In [35]:
#df2022.to_csv("TS2022.csv", index=False)